In [6]:
##First Kernel using python - interested for your feedback to where I went wrong.
import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import base64
import string
import re
from collections import Counter
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
##I manually add the missing column - why??


In [7]:
train.head()

,label,sentence
0,y,"Lediglich ergänzend wird angemerkt, dass nicht..."
1,y,Die Inzidentkontrolle ist auch nicht nach § 47...
2,y,In Bezug auf die „Prüfungsdichte“ orientiert s...
3,y,Aus der Gesamtheit der Urteilsgründe ergibt si...
4,n,Nach den Geschäftsbedingungen von eBay (Bl. 59...


In [8]:

#test['project_is_approved'] = 1

import spacy

nlp = spacy.load('de_core_news_sm')
punctuations = string.punctuation

# Define function to cleanup text by removing personal pronouns, stopwords, and puncuation
def cleanup_text(docs, logging=False):
    texts = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents." % (counter, len(docs)))
        counter += 1
        doc = nlp(doc)
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
        tokens = [tok for tok in tokens if tok not in punctuations and tok not in stopwords]
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return pd.Series(texts)
    
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from nltk.corpus import stopwords
import string
import re
import spacy
#spacy.load('en')
from spacy.lang.de import German
parser = German()

STOPLIST = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS))
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”"]


class CleanTextTransformer(TransformerMixin):
    def transform(self, X, **transform_params):
        return [cleanText(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}
    
def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = text.lower()
    return text

def tokenizeText(sample):
    tokens = parser(sample)
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    tokens = [tok for tok in tokens if tok not in STOPLIST]
    tokens = [tok for tok in tokens if tok not in SYMBOLS]
    return tokens

def printNMostInformative(vectorizer, clf, N):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    topClass1 = coefs_with_fns[:N]
    topClass2 = coefs_with_fns[:-(N + 1):-1]
    print("Class 1 best: ")
    for feat in topClass1:
        print(feat)
    print("Class 2 best: ")
    for feat in topClass2:
        print(feat)

vectorizer = CountVectorizer(tokenizer=tokenizeText, ngram_range=(1,1))
clf = LinearSVC()
pipe = Pipeline([('cleanText', CleanTextTransformer()), ('vectorizer', vectorizer), ('clf', clf)])


# data
##train = train[0:10000]
train1 = train['sentence'].tolist()
labelsTrain1 = train['label'].tolist()

##test = test[0:10000]
test1 = test['sentence'].tolist()
labelsTest1 = test['label'].tolist()

pipe.fit(train1, labelsTrain1)

# test
preds = pipe.predict(test1)

print("accuracy:", accuracy_score(labelsTest1, preds))
print("Top 10 features used to predict: ")

printNMostInformative(vectorizer, clf, 10)

pipe = Pipeline([('cleanText', CleanTextTransformer()), ('vectorizer', vectorizer)])
transform = pipe.fit_transform(train1, labelsTrain1)
vocab = vectorizer.get_feature_names()

for i in range(len(train1)):
    s = ""
    indexIntoVocab = transform.indices[transform.indptr[i]:transform.indptr[i+1]]
    numOccurences = transform.data[transform.indptr[i]:transform.indptr[i+1]]
    for idx, num in zip(indexIntoVocab, numOccurences):
        s += str((vocab[idx], num))

from sklearn import metrics

#print(metrics.classification_report(labelsTest1, preds, 
#                                    target_names=test['school_state'].unique()))

accuracy: 0.05263157894736842
Top 10 features used to predict: 
Class 1 best: 
(-0.258041680932354, 'nur')
(-0.22134252197199356, 'nicht')
(-0.1859825502822753, 'dürfen')
(-0.135950470174137, 'welch')
(-0.1340921555494968, 'wenig')
(-0.13263949693176716, 'urteil')
(-0.1273110523404185, 'a.')
(-0.12656134512687622, 'fall')
(-0.11919233055987, 'danach')
(-0.11667332397113697, 'allgemein')
Class 2 best: 
(0.22567303484074472, 'einen')
(0.15419441012957186, 'einer')
(0.14388190933209466, 'als')
(0.12645309160794177, 'parteien')
(0.11492954622851492, '§')
(0.09567694027998948, 'ein')
(0.09423537896406292, '10')
(0.088022773482925, 'ich')
(0.08530286724862787, 'vergleiche')
(0.07860961363255267, 'auf')


In [9]:
pipe.predict('foooo')

AttributeError: 'CountVectorizer' object has no attribute 'predict'

In [65]:
#!/usr/bin/env python
# coding: utf8
"""Train a convolutional neural network text classifier on the
IMDB dataset, using the TextCategorizer component. The dataset will be loaded
automatically via Thinc's built-in dataset loader. The model is added to
spacy.pipeline, and predictions are available via `doc.cats`. For more details,
see the documentation:
* Training: https://spacy.io/usage/training

Compatible with: spaCy v2.0.0+
"""
import plac
import random
from pathlib import Path
import thinc.extra.datasets

import spacy
from spacy.util import minibatch, compounding

def load_data(limit=0, split=0.8):
    """Load data from the IMDB dataset."""
    # Partition off part of the train data for evaluation
    train_data, _ = thinc.extra.datasets.imdb()
    random.shuffle(train_data)
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats = [{'POSITIVE': bool(y)} for y in labels]
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])


def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 0.0   # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0   # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score}



In [12]:
nlp = spacy.load('de_core_news_sm')


n_iter=20
n_texts=2000


# add the text classifier to the pipeline if it doesn't exist
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'textcat' not in nlp.pipe_names:
    textcat = nlp.create_pipe('textcat')
    nlp.add_pipe(textcat, last=True)
# otherwise, get it, so we can add labels to it
else:
    textcat = nlp.get_pipe('textcat')

# add label to text classifier
textcat.add_label('POSITIVE')



Loading IMDB data...
Using 2000 examples (1600 training, 400 evaluation)


In [14]:
train_data

train_data_ = [
    ('foooo', {'cats': {'POSITIVE': True}}),
    ('foooo', {'cats': {'POSITIVE': True}}),
    ('foooo', {'cats': {'POSITIVE': True}}),
    ('foooo', {'cats': {'POSITIVE': True}}),
    ('foooo', {'cats': {'POSITIVE': True}}),
    ('foooo', {'cats': {'POSITIVE': True}}),
    ('foooo', {'cats': {'POSITIVE': True}}),
    ('foooo', {'cats': {'POSITIVE': True}}),
    ('foooo', {'cats': {'POSITIVE': True}}),
    ('foooo', {'cats': {'POSITIVE': True}}),
    
]

In [77]:
"""
Definition.txt  Ergebnis.txt  hackathon2019-parse.py  LICENSE  README.md  Rechtsansicht.txt  
Streitbezogen.txt  Subsumtion.txt  Tatbestand.txt
"""
with open('autocom/Definition.txt') as f:
    definition = f.readlines()

with open('autocom/Ergebnis.txt') as f:
    ergebnis = f.readlines()

with open('autocom/Rechtsansicht.txt') as f:
    rechtsansicht = f.readlines()
    
with open('autocom/Streitbezogen.txt') as f:
    positive = f.readlines()

    
data = []

data.extend(list(zip(definition, [{'cat': {'POSITIVE': True}} for y in range(len(definition))])))

data.extend(list(zip(ergebnis, [{'cat': {'POSITIVE': False}} for y in range(len(ergebnis))])))



split=0.8
random.shuffle(data)
split = int(len(data) * split)

train_data = data[:split]
test_data = data[split:]

test_text = [x[0] for x in test_data]
test_text

test_label = [x[1]['cat'] for x in test_data]
test_label

[{'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POS

In [81]:
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(limit=n_texts)

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()
    print("Training the model...")
    print('{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F'))
    for i in range(n_iter):
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            
            #print(texts)
            #print(annotations)
            
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
        with textcat.model.use_params(optimizer.averages):
            # evaluate on the dev data split off in load_data()
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
            #scores = evaluate(nlp.tokenizer, textcat, test_text, test_label)
            
            
        print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}'  # print a simple table
              .format(losses['textcat'], scores['textcat_p'],
                      scores['textcat_r'], scores['textcat_f']))

# test the trained model
test_text = "This movie sucked"
doc = nlp(test_text)
print(test_text, doc.cats)


Training the model...
LOSS 	  P  	  R  	  F  


TypeError: __init__() got an unexpected keyword argument 'cat'

In [45]:
sent = 'Nach § 51 Abs.3 VwVfG muss der Antrag binnen drei Monaten seit dem Tag gestellt werden, an dem der Betroffene von dem Grund für das Wiederaufgreifen Kenntnis erhalten hat.'
sent = 'Die Klägerin trägt die Kosten des Verfahrens.'

x = nlp(sent)

In [46]:
x.cats

{'POSITIVE': 0.7044544219970703}

In [59]:
    print("Loading IMDB data...")
    (train_texts, train_cats), (dev_texts, dev_cats) = load_data(limit=n_texts)
    print("Using {} examples ({} training, {} evaluation)"
          .format(n_texts, len(train_texts), len(dev_texts)))
    train_data = list(zip(train_texts,
[{'cats': cats} for cats in train_cats]))

Loading IMDB data...
Using 2000 examples (1600 training, 400 evaluation)


In [61]:
train_data[0]

('Val Kilmer and Dylan McDermott are terrific. I have seen Kilmer on The Doors, however his interpretation of John Holmes is superb. Nothing compared to Boogie Nights which was kind of slow. Wonderland is a movie which is able to show you a horrible crime story from the perspective from a guy who is just indulged in his drug vice and indolent of what ever happens around. At the same time, the John Holmes character shows a very clever hustler who is able to pass through the nastiest and ugliest situations almost unharmed. The movie deserves being watched more than once. The seventies ambiance sensual and full of drugs is amazing.',
 {'cats': {'POSITIVE': True}})

In [62]:
test_data[0]

('Die Annahme der Auflassungserklärung stellt aber, wie ausgeführt, keine Verfügung über den Nachlass dar.\n',
 {'cat': {'POSITIVE': False}})

In [68]:
dev_texts

("I can only agree with taximeter that this is a fantastic film and should be seen by a wide audience. The imagination on display, the visual interpretation of the script, the humor is constantly surprising. The two leads are great and really carry the film. My advice would be to not even watch a trailer, just rent the film and watch without expectations. I rented from blockbuster, so it is readily available in brisbane, not everyone will enjoy it but i think most people will have an opinion and that's always good, unless it's just 'that was stupid'. I loved this film, you just don't get to see gem's like this every day. This should become a cult favorite. Give it a try, you may just feel the same way about it as i do.",
 '"The Groove Tube" was one of only two Ken Shapiro movies, the other one being the equally zany "Modern Problems". This one is just a full-scale parody of TV. Aside from Shapiro - who apparently didn\'t do anything after "Modern Problems" - the movie also stars Chevy 

In [69]:
dev_cats

[{'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITI

[{'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': False},
 {'POSITIVE': True},
 {'POSITIVE': False},
 {'POSITIV